###  Forward pass check

In [1]:
import torch
data = torch.randn(10, 5)

featmap_H, featmap_W = 4, 4 

feat_lang = data.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, featmap_H, featmap_W)

# 10, 4, 4, 5
# 10, 5, 4, 4

In [2]:
feat_lang.size()

torch.Size([10, 5, 4, 4])

In [13]:
from model import ImgSegRefExpModel
model = ImgSegRefExpModel(mlp_hidden=5, vocab_size=10, emb_size=9, lstm_hidden_size=7)

In [6]:
data = (torch.randn(2, 3, 512, 512), torch.randint(low=0, high=9, size=(2, 5)))

In [7]:
out = model(data)
out.shape

torch.Size([2, 1000, 16, 16])
16 16
torch.Size([2, 1, 16, 16])


## Demo for Pytorch implementation of baseline

In [9]:
from utils import *
vocab_file = 'vocabulary_referit.txt'
# Load vocabulary
vocab_dict = load_vocab_dict_from_file(vocab_file)

In [11]:
# input image and query text
# Just need file name. Everything else is auto-magic
new_file = '10002.jpg'  #@param {type:"string"}
query = 'water' #@param {type:"string"}

In [20]:
from model import ImgSegRefExpModel
pretrained_model_file = "text_objseg_pretrained_torch_converted_with_lstm"

model = ImgSegRefExpModel(mlp_hidden=500, vocab_size=8803, emb_size=1000, lstm_hidden_size=1000)
model.load_state_dict(torch.load(pretrained_model_file))

RuntimeError: Error(s) in loading state_dict for ImgSegRefExpModel:
	Missing key(s) in state_dict: "mlp1.0.weight", "mlp2.0.weight". 
	Unexpected key(s) in state_dict: "mlp.0.weight", "mlp.0.bias", "mlp.1.weight", "mlp.1.bias". 
	size mismatch for img_features.feature_extractor.vgg_fc7_full_conv.0.0.weight: copying a param with shape torch.Size([4096, 25088]) from checkpoint, the shape in current model is torch.Size([4096, 512, 7, 7]).
	size mismatch for img_features.feature_extractor.vgg_fc7_full_conv.1.0.weight: copying a param with shape torch.Size([4096, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 4096, 1, 1]).
	size mismatch for img_features.feature_extractor.vgg_fc8_full_conv.weight: copying a param with shape torch.Size([1000, 4096]) from checkpoint, the shape in current model is torch.Size([1000, 4096, 1, 1]).

In [17]:
model.state_dict().keys()

odict_keys(['text_features.embedding.weight', 'text_features.lstm.weight_ih_l0', 'text_features.lstm.weight_hh_l0', 'text_features.lstm.bias_ih_l0', 'text_features.lstm.bias_hh_l0', 'img_features.feature_extractor.0.weight', 'img_features.feature_extractor.0.bias', 'img_features.feature_extractor.2.weight', 'img_features.feature_extractor.2.bias', 'img_features.feature_extractor.5.weight', 'img_features.feature_extractor.5.bias', 'img_features.feature_extractor.7.weight', 'img_features.feature_extractor.7.bias', 'img_features.feature_extractor.10.weight', 'img_features.feature_extractor.10.bias', 'img_features.feature_extractor.12.weight', 'img_features.feature_extractor.12.bias', 'img_features.feature_extractor.14.weight', 'img_features.feature_extractor.14.bias', 'img_features.feature_extractor.17.weight', 'img_features.feature_extractor.17.bias', 'img_features.feature_extractor.19.weight', 'img_features.feature_extractor.19.bias', 'img_features.feature_extractor.21.weight', 'img_fea

In [19]:
file = "text_objseg_pretrained_torch_converted_with_lstm"
torch.load(file).keys()

odict_keys(['img_features.feature_extractor.0.weight', 'img_features.feature_extractor.0.bias', 'img_features.feature_extractor.2.weight', 'img_features.feature_extractor.2.bias', 'img_features.feature_extractor.5.weight', 'img_features.feature_extractor.5.bias', 'img_features.feature_extractor.7.weight', 'img_features.feature_extractor.7.bias', 'img_features.feature_extractor.10.weight', 'img_features.feature_extractor.10.bias', 'img_features.feature_extractor.12.weight', 'img_features.feature_extractor.12.bias', 'img_features.feature_extractor.14.weight', 'img_features.feature_extractor.14.bias', 'img_features.feature_extractor.17.weight', 'img_features.feature_extractor.17.bias', 'img_features.feature_extractor.19.weight', 'img_features.feature_extractor.19.bias', 'img_features.feature_extractor.21.weight', 'img_features.feature_extractor.21.bias', 'img_features.feature_extractor.24.weight', 'img_features.feature_extractor.24.bias', 'img_features.feature_extractor.26.weight', 'img_f

In [48]:
import os
im_file = new_file

# Run on the input image and query text
text_seq_val = np.zeros((T, N), dtype=np.float32)
imcrop_val = np.zeros((N, input_H, input_W, 3), dtype=np.float32)

# Preprocess image and text
im = skimage.io.imread(im_file)
processed_im = skimage.img_as_ubyte(im_processing.resize_and_pad(im, input_H, input_W))
imcrop_val[0, :] = processed_im.astype(np.float32) - segmodel.vgg_net.channel_mean
text_seq_val[:, 0] = utils.preprocess_sentence(query, vocab_dict, T)

# Forward pass to get response map
scores_val = np.squeeze(sess.run(scores, feed_dict={text_seq_batch: text_seq_val,
                                                    imcrop_batch: imcrop_val}))

output = model((imcrop_val, text_seq_val))

# Final prediction
prediction = im_processing.resize_and_crop(scores_val>0, *im.shape[:2]).astype(np.bool)
print(prediction)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(prediction)

print("query text = '%s'" % query)

NameError: name 'T' is not defined